In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import pymongo
import cx_Oracle


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [2]:
conn_str = u'report/justd0it@10.234.152.61:1521/alfabi'
connection = cx_Oracle.connect(conn_str)

conn_mongo = pymongo.MongoClient\
        ("mongodb://127.0.0.1:27017/seq_trans")
mydb = conn_mongo["seq_trans"]
print(mydb.list_collection_names())

mycol = mydb['member_item_seq']

['member_item_seq']


In [3]:
# df_sgm = pd.read_csv('df_trans_user_item_date.csv', sep='\t', dtype='object')
# df_item = pd.read_csv('MASTER_ITEM_202012121853.csv', dtype='object')
# df_sgm['TR_DATE'] = df_sgm['TR_DATE'].map(lambda x: eval(x))
# df_sgm['TR_PLU'] = df_sgm['TR_PLU'].apply(lambda x: x[0:-1])

In [4]:
q_test = """
SELECT * FROM report.RPT_TRANS rt 
WHERE rt.TR_MEMBERS = '9990011847758192'
AND rt.TR_PLU_PCS = '120763'
AND TR_DATE BETWEEN '11nov20' AND '11nov20'

"""

df_test = pd.read_sql_query(q_test, connection)   

# df_test.groupby(['TR_MEMBERS','TR_PLU_PCS','TR_DATE']).agg({'TR_QTY':'sum'})
df_test[['TR_MEMBERS','TR_PLU_PCS','TR_DATE','TR_QTY']]

,TR_MEMBERS,TR_PLU_PCS,TR_DATE,TR_QTY
0,9990011847758192,120763,2020-11-11,2.0
1,9990011847758192,120763,2020-11-11,1.0
2,9990011847758192,120763,2020-11-11,1.0
3,9990011847758192,120763,2020-11-11,4.0


In [5]:
mycol.find_one({'TR_MEMBERS':'9990011847758192', 'TR_PLU_PCS':120763})

{'_id': ObjectId('5fd90c05b8f8002aaad754f7'),
 'TR_MEMBERS': '9990011847758192',
 'TR_PLU_PCS': 120763,
 'updated_at': datetime.datetime(2021, 1, 13, 2, 13, 37, 643000),
 'TRX_DATE': {'1607904000': [3, '- -'],
  '1605830400': [2, '-'],
  '1605398400': [1, '-'],
  '1605139200': [3, '- -'],
  '1605052800': [8, '- - - -'],
  '1609545600': [3, '-']},
 'TRF_DATE': {'1609545600': 3,
  '1607904000': 3,
  '1605830400': 2,
  '1605398400': 1,
  '1605139200': 3,
  '1605052800': 8}}

In [6]:
# q_plu = """
# SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
# where lower(mpa.MPA_DESC) like 'sgm eksplor%'
# """

# df_plu = pd.read_sql_query(q_plu, connection)     
# li_plu = df_plu['MPA_PLU']


# df_all = []
# for plu in li_plu[0:20]:
#     df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
#     df_all.append(df)

# df_seq_1 = pd.concat(df_all)
# df_seq_1.head()
df_seq_1 = pd.DataFrame()

In [7]:
q_plu = """
SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
where lower(mpa.MPA_DESC) like 'sgm eksplor%'
or lower(mpa.MPA_DESC) like 'anlene%'
or lower(mpa.MPA_DESC) like 'dancow%'
or lower(mpa.MPA_DESC) like 'bebelac%'
or lower(mpa.MPA_DESC) like 'lactogrow%'
or lower(mpa.MPA_DESC) like 'ff baby%'
or lower(mpa.MPA_DESC) like 'pepsodent pg economy%'
"""

df_plu = pd.read_sql_query(q_plu, connection)     
li_plu = df_plu['MPA_PLU']


df_all = []
for plu in li_plu:
    df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
    df_all.append(df)

df_seq_2 = pd.concat(df_all)
df_seq_2.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,TRF_DATE,last_order_date,next_order_date
0,5fd90a51b8f8002aaaba19da,9990013140543181,120763,2021-01-12 13:56:04.496,"{'1607904000': [1, '-'], '1607558400': [1, '-'], '1603929600': 1, '1609804800': [1, '-']}","{'1607904000': 1, '1607558400': 1, '1603929600': 1, '1609804800': 1}",NaT,NaT
1,5fd90a51b8f8002aaaba19e7,9990011494110271,120763,2021-01-12 13:19:07.981,"{'1607904000': [1, '-'], '1597276800': 1}","{'1607904000': 1, '1597276800': 1}",NaT,NaT
2,5fd90a51b8f8002aaaba2495,9990021000743963,120763,2021-01-12 13:20:05.987,"{'1607904000': [1, '-']}",{'1607904000': 1},NaT,NaT
3,5fd90a51b8f8002aaaba3bb2,9990011419048278,120763,2021-01-12 17:55:48.841,"{'1607990400': [1, '-'], '1607904000': [1, '-'], '1605398400': [1, '-'], '1602201600': 1}","{'1607990400': 1, '1607904000': 1, '1605398400': 1, '1602201600': 1}",NaT,NaT
4,5fd90a52b8f8002aaaba5d59,9990012150859676,120763,2021-01-12 13:20:26.659,"{'1607904000': [1, '-'], '1603929600': 1, '1594684800': 1, '1591142400': 1, '1588896000': 1, '1588550400': 1, '1588204800': 1, '1588032000': 1, '1587859200': 1, '1587427200': 1, '1586908800': 1, '1609286400': [1, '-']}","{'1609286400': 1, '1607904000': 1, '1603929600': 1, '1594684800': 1}",NaT,NaT


In [8]:
df_seq = pd.concat([df_seq_1, df_seq_2]).reset_index(drop=True)

In [9]:
q_nas = """
SELECT ITEM_N, DESCP, UKURAN, PANJANG, LEBAR, TINGGI, UOM FROM MASTER_NAS_M mnm 
"""

df_nas = pd.read_sql_query(q_nas, connection)
display(df_nas.head())

def norm_uom(uom):
    li_uom_norm = []
    li_weight = []
    
    if uom == np.nan:
        li_weight.append(np.nan)
        li_uom_norm.append(np.nan)
    elif uom == 'KG':
        li_weight.append(1000)
        li_uom_norm.append('G')
    elif uom == 'L':
        li_weight.append(1000)
        li_uom_norm.append('ML')
    else:
        li_weight.append(1)
        li_uom_norm.append(uom)

    return li_uom_norm[0], li_weight[0]
            
        
df_nas['UOM_NORM'], df_nas['weight'] = zip(*df_nas['UOM'].apply(norm_uom))
df_nas['UKURAN_NORM'] = df_nas['UKURAN'] * df_nas['weight']


df_nas['UOM'] = df_nas['UOM_NORM']
df_nas['UKURAN']= df_nas['UKURAN_NORM']

,ITEM_N,DESCP,UKURAN,PANJANG,LEBAR,TINGGI,UOM
0,851706,SAUCEPAN 16CM (SN-0211),NaN,1.0,1.0,1.0,None
1,851707,CYPRUZ AIR FRYER 4.5L (DF-0155),NaN,1.0,1.0,1.0,None
2,851708,CYPRUZ GRIL & STEAMBOT(TG-0295),NaN,1.0,1.0,1.0,None
3,851709,CYPRUZ COOKWARE SET 4 (PI-0900),NaN,1.0,1.0,1.0,None
4,851710,CYPRUZ COOKWARE SET 5 (PI-0901),NaN,1.0,1.0,1.0,None


In [10]:
df = pd.merge(df_seq, df_nas, left_on='TR_PLU_PCS', right_on='ITEM_N')[list(df_seq) + ['DESCP','UKURAN','UOM']]
df = df[df['TRF_DATE'].notnull()]

min_freq = 3
df_hist = df[df['TRF_DATE'].apply(lambda x: len(x)) >= min_freq]

In [11]:
df_hist.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,TRF_DATE,last_order_date,next_order_date,DESCP,UKURAN,UOM
0,5fd90a51b8f8002aaaba19da,9990013140543181,120763,2021-01-12 13:56:04.496,"{'1607904000': [1, '-'], '1607558400': [1, '-'], '1603929600': 1, '1609804800': [1, '-']}","{'1607904000': 1, '1607558400': 1, '1603929600': 1, '1609804800': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
3,5fd90a51b8f8002aaaba3bb2,9990011419048278,120763,2021-01-12 17:55:48.841,"{'1607990400': [1, '-'], '1607904000': [1, '-'], '1605398400': [1, '-'], '1602201600': 1}","{'1607990400': 1, '1607904000': 1, '1605398400': 1, '1602201600': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
4,5fd90a52b8f8002aaaba5d59,9990012150859676,120763,2021-01-12 13:20:26.659,"{'1607904000': [1, '-'], '1603929600': 1, '1594684800': 1, '1591142400': 1, '1588896000': 1, '1588550400': 1, '1588204800': 1, '1588032000': 1, '1587859200': 1, '1587427200': 1, '1586908800': 1, '1609286400': [1, '-']}","{'1609286400': 1, '1607904000': 1, '1603929600': 1, '1594684800': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
5,5fd90a52b8f8002aaaba5e94,9990011323821648,120763,2021-01-12 14:21:54.623,"{'1607904000': [1, '-'], '1607472000': [1, '-'], '1607299200': [1, '-'], '1609027200': [1, '-'], '1610323200': [1, '-'], '1609113600': [1, '-']}","{'1609113600': 1, '1609027200': 1, '1607904000': 1, '1607472000': 1, '1607299200': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
11,5fd90a53b8f8002aaabace0b,9990013066618077,120763,2021-01-12 14:51:36.553,"{'1608336000': [1, '-'], '1608076800': [1, '-'], '1607904000': [1, '-'], '1607817600': [1, '-'], '1607731200': [1, '-'], '1607126400': [1, '-'], '1607040000': [1, '-']}","{'1608336000': 1, '1608076800': 1, '1607904000': 1, '1607817600': 1, '1607731200': 1, '1607126400': 1, '1607040000': 1}",2020-12-19,2020-12-22,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR


In [12]:
# set_time = {'1608163200': 1, '1607904000': 1, '1607040000': 1, '1606176000': 1, '1605916800': 1, '1605657600': 1, '1605225600': 1, '1602288000': 1}
# set_time = sorted(set_time.items())


# print(set_time)

In [13]:
def cal_feature(set_timestamp_old):

    set_timestamp_old = sorted(set_timestamp_old.items())
    last_order = int(set_timestamp_old[-1][0])
        
    read_seq = []
    qty_seq = []
    li_seq = []
    set_timestamp = []
    for set_t in (set_timestamp_old):
        if isinstance((set_t[1]), int):

            continue
        set_timestamp.append(set_t)
    print(set_timestamp)
    
    if len(set_timestamp) < min_freq:
        print('not enough')
        return 0, 0, 0
    
    for idx in range(0,len(set_timestamp),1):

        if str(set_timestamp[idx][1][-1]).startswith('-'):
            print('masuk')
            # convert to readable timestamp and append quantity
            read_seq.append(datetime.fromtimestamp(int(set_timestamp[idx][0])).strftime('%d%b%y'))
            qty_seq.append(set_timestamp[idx][1][0])

            # find different day between order 
            # if (idx+1) < len(set_timestamp):
            diff = int(set_timestamp[idx][0]) - int(set_timestamp[idx-1][0])
            diff_day = int((diff / 3600) / 24)
            li_seq.append(diff_day)
        

    print(li_seq)
    if len(li_seq) < min_freq-1:
        print('not enough')
        return 0, 0, 0
    qty_seq = qty_seq[-1*(min_freq):-1]
    
    # get onyly until needed sequence order
    li_seq = li_seq[-1*(min_freq-1):]
    # max diff day
    seq_max = np.max(li_seq)
    # min diff day
    seq_min = np.min(li_seq)
    # day since
    seq_since = li_seq[-2]
    # mean
    seq_mean = round(np.mean(li_seq),2)
    # mean_qty
    seq_mean_qty = round(np.mean([li_seq[i]/qty_seq[i] for i in range(len(li_seq))]),2)
    # variance
    seq_var = round(np.var(li_seq),2)

    ## appending
    li_seq.append(seq_max)
    li_seq.append(seq_min)
    li_seq.append(seq_since)
    li_seq.append(seq_mean)
    li_seq.append(seq_var)
    
    li_seq.extend(qty_seq)
    li_seq.append(seq_mean_qty)


    return li_seq, last_order, read_seq

In [14]:
def cal_feature_old(set_timestamp):

    set_timestamp = sorted(set_timestamp.items())
    last_order = int(set_timestamp[-1][0])
        
    read_seq = []
    qty_seq = []
    li_seq = []

    for idx in range(0,len(set_timestamp),1):

            # convert to readable timestamp and append quantity
            read_seq.append(datetime.fromtimestamp(int(set_timestamp[idx][0])).strftime('%d%b%y'))
            qty_seq.append(set_timestamp[idx][1])

            # find different day between order 
            # if (idx+1) < len(set_timestamp):
            diff = int(set_timestamp[idx][0]) - int(set_timestamp[idx-1][0])
            diff_day = int((diff / 3600) / 24)
            li_seq.append(diff_day)
        
    
    qty_seq = qty_seq[-1*(min_freq):-1]
    
    # get onyly until needed sequence order
    li_seq = li_seq[-1*(min_freq-1):]
    # max diff day
    seq_max = np.max(li_seq)
    # min diff day
    seq_min = np.min(li_seq)
    # day since
    seq_since = li_seq[-2]
    # mean
    seq_mean = round(np.mean(li_seq),2)
    # mean_qty
    seq_mean_qty = round(np.mean([li_seq[i]/qty_seq[i] for i in range(len(li_seq))]),2)
    # variance
    seq_var = round(np.var(li_seq),2)

    ## appending
    li_seq.append(seq_max)
    li_seq.append(seq_min)
    li_seq.append(seq_since)
    li_seq.append(seq_mean)
    li_seq.append(seq_var)
    
    li_seq.extend(qty_seq)
    li_seq.append(seq_mean_qty)


    return li_seq, last_order, read_seq

In [15]:
# df_hist['tr_date_seq'], df_hist['last_order'], df_hist['read_seq'] = zip(*df_hist['TRX_DATE'].apply(cal_feature))

In [16]:
df_hist['tr_date_seq'], df_hist['last_order'], df_hist['read_seq'] = zip(*df_hist['TRF_DATE'].apply(cal_feature_old))

<ipython-input-16-e6c739f2e245>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hist['tr_date_seq'], df_hist['last_order'], df_hist['read_seq'] = zip(*df_hist['TRF_DATE'].apply(cal_feature_old))
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [17]:
df_hist.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,TRF_DATE,last_order_date,next_order_date,DESCP,UKURAN,UOM,tr_date_seq,last_order,read_seq
0,5fd90a51b8f8002aaaba19da,9990013140543181,120763,2021-01-12 13:56:04.496,"{'1607904000': [1, '-'], '1607558400': [1, '-'], '1603929600': 1, '1609804800': [1, '-']}","{'1607904000': 1, '1607558400': 1, '1603929600': 1, '1609804800': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(4, 22, 22, 4, 4, 13.0, 81.0, 1, 1, 13.0)",1609804800,"[29Oct20, 10Dec20, 14Dec20, 05Jan21]"
3,5fd90a51b8f8002aaaba3bb2,9990011419048278,120763,2021-01-12 17:55:48.841,"{'1607990400': [1, '-'], '1607904000': [1, '-'], '1605398400': [1, '-'], '1602201600': 1}","{'1607990400': 1, '1607904000': 1, '1605398400': 1, '1602201600': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(29, 1, 29, 1, 29, 15.0, 196.0, 1, 1, 15.0)",1607990400,"[09Oct20, 15Nov20, 14Dec20, 15Dec20]"
4,5fd90a52b8f8002aaaba5d59,9990012150859676,120763,2021-01-12 13:20:26.659,"{'1607904000': [1, '-'], '1603929600': 1, '1594684800': 1, '1591142400': 1, '1588896000': 1, '1588550400': 1, '1588204800': 1, '1588032000': 1, '1587859200': 1, '1587427200': 1, '1586908800': 1, '1609286400': [1, '-']}","{'1609286400': 1, '1607904000': 1, '1603929600': 1, '1594684800': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(46, 16, 46, 16, 46, 31.0, 225.0, 1, 1, 31.0)",1609286400,"[14Jul20, 29Oct20, 14Dec20, 30Dec20]"
5,5fd90a52b8f8002aaaba5e94,9990011323821648,120763,2021-01-12 14:21:54.623,"{'1607904000': [1, '-'], '1607472000': [1, '-'], '1607299200': [1, '-'], '1609027200': [1, '-'], '1610323200': [1, '-'], '1609113600': [1, '-']}","{'1609113600': 1, '1609027200': 1, '1607904000': 1, '1607472000': 1, '1607299200': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(13, 1, 13, 1, 13, 7.0, 36.0, 1, 1, 7.0)",1609113600,"[07Dec20, 09Dec20, 14Dec20, 27Dec20, 28Dec20]"
11,5fd90a53b8f8002aaabace0b,9990013066618077,120763,2021-01-12 14:51:36.553,"{'1608336000': [1, '-'], '1608076800': [1, '-'], '1607904000': [1, '-'], '1607817600': [1, '-'], '1607731200': [1, '-'], '1607126400': [1, '-'], '1607040000': [1, '-']}","{'1608336000': 1, '1608076800': 1, '1607904000': 1, '1607817600': 1, '1607731200': 1, '1607126400': 1, '1607040000': 1}",2020-12-19,2020-12-22,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(2, 3, 3, 2, 2, 2.5, 0.25, 1, 1, 2.5)",1608336000,"[04Dec20, 05Dec20, 12Dec20, 13Dec20, 14Dec20, 16Dec20, 19Dec20]"


In [18]:
#col_feature = ['t_5', 't_4', 't_3', 't_2', 't_1', 'max', 'min', 'since', 'mean', 'variance', 'q_5', 'q_4', 'q_3', 'q_2', 'q_1', 'mean_qty']
col_feature = ['t_'+ str(i) for i in range((min_freq-1),0,-1)] + \
                ['max', 'min', 'since', 'mean', 'variance'] + \
                ['q_'+ str(i) for i in range((min_freq-1),0,-1)] + ['mean_qty']
train = pd.DataFrame(df_hist.tr_date_seq.tolist(), index=df_hist.index, columns=col_feature)

In [19]:
df_hist.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,TRF_DATE,last_order_date,next_order_date,DESCP,UKURAN,UOM,tr_date_seq,last_order,read_seq
0,5fd90a51b8f8002aaaba19da,9990013140543181,120763,2021-01-12 13:56:04.496,"{'1607904000': [1, '-'], '1607558400': [1, '-'], '1603929600': 1, '1609804800': [1, '-']}","{'1607904000': 1, '1607558400': 1, '1603929600': 1, '1609804800': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(4, 22, 22, 4, 4, 13.0, 81.0, 1, 1, 13.0)",1609804800,"[29Oct20, 10Dec20, 14Dec20, 05Jan21]"
3,5fd90a51b8f8002aaaba3bb2,9990011419048278,120763,2021-01-12 17:55:48.841,"{'1607990400': [1, '-'], '1607904000': [1, '-'], '1605398400': [1, '-'], '1602201600': 1}","{'1607990400': 1, '1607904000': 1, '1605398400': 1, '1602201600': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(29, 1, 29, 1, 29, 15.0, 196.0, 1, 1, 15.0)",1607990400,"[09Oct20, 15Nov20, 14Dec20, 15Dec20]"
4,5fd90a52b8f8002aaaba5d59,9990012150859676,120763,2021-01-12 13:20:26.659,"{'1607904000': [1, '-'], '1603929600': 1, '1594684800': 1, '1591142400': 1, '1588896000': 1, '1588550400': 1, '1588204800': 1, '1588032000': 1, '1587859200': 1, '1587427200': 1, '1586908800': 1, '1609286400': [1, '-']}","{'1609286400': 1, '1607904000': 1, '1603929600': 1, '1594684800': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(46, 16, 46, 16, 46, 31.0, 225.0, 1, 1, 31.0)",1609286400,"[14Jul20, 29Oct20, 14Dec20, 30Dec20]"
5,5fd90a52b8f8002aaaba5e94,9990011323821648,120763,2021-01-12 14:21:54.623,"{'1607904000': [1, '-'], '1607472000': [1, '-'], '1607299200': [1, '-'], '1609027200': [1, '-'], '1610323200': [1, '-'], '1609113600': [1, '-']}","{'1609113600': 1, '1609027200': 1, '1607904000': 1, '1607472000': 1, '1607299200': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(13, 1, 13, 1, 13, 7.0, 36.0, 1, 1, 7.0)",1609113600,"[07Dec20, 09Dec20, 14Dec20, 27Dec20, 28Dec20]"
11,5fd90a53b8f8002aaabace0b,9990013066618077,120763,2021-01-12 14:51:36.553,"{'1608336000': [1, '-'], '1608076800': [1, '-'], '1607904000': [1, '-'], '1607817600': [1, '-'], '1607731200': [1, '-'], '1607126400': [1, '-'], '1607040000': [1, '-']}","{'1608336000': 1, '1608076800': 1, '1607904000': 1, '1607817600': 1, '1607731200': 1, '1607126400': 1, '1607040000': 1}",2020-12-19,2020-12-22,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR,"(2, 3, 3, 2, 2, 2.5, 0.25, 1, 1, 2.5)",1608336000,"[04Dec20, 05Dec20, 12Dec20, 13Dec20, 14Dec20, 16Dec20, 19Dec20]"


In [20]:
df_hist_cond = df_hist.loc[list(train[(train['mean_qty'] > 7) & (train['variance'] < 80)].index)]
df_hist_cond[df_hist_cond['TR_PLU_PCS'] == 120771]

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,TRF_DATE,last_order_date,next_order_date,DESCP,UKURAN,UOM,tr_date_seq,last_order,read_seq
734763,5fd90a51b8f8002aaaba2f27,9990012076527765,120771,2021-01-13 03:26:07.656,"{'1607904000': [1, '-'], '1606435200': [1, '-'], '1604620800': [1, '-'], '1595462400': 1, '1595030400': 1, '1593216000': 1, '1591660800': 1, '1590192000': 1, '1589760000': 1, '1589414400': 1, '1588982400': 1, '1588204800': 1, '1586908800': 1, '1586563200': 1}","{'1607904000': 1, '1606435200': 1, '1604620800': 1, '1595462400': 1, '1595030400': 1}",2020-12-14,2021-01-14,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(21, 17, 21, 17, 21, 19.0, 4.0, 1, 1, 19.0)",1607904000,"[18Jul20, 23Jul20, 06Nov20, 27Nov20, 14Dec20]"
734772,5fd90a53b8f8002aaabab722,9990012150914218,120771,2021-01-13 04:23:34.172,"{'1608336000': [1, '-'], '1607904000': [1, '-'], '1606780800': [1, '-'], '1604880000': [1, '-'], '1604188800': [1, '-'], '1594684800': 1, '1592697600': 1}","{'1608336000': 1, '1607904000': 1, '1606780800': 1, '1604880000': 1, '1604188800': 1, '1594684800': 1}",2020-12-19,2021-01-15,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(13, 5, 13, 5, 13, 9.0, 16.0, 1, 1, 9.0)",1608336000,"[14Jul20, 01Nov20, 09Nov20, 01Dec20, 14Dec20, 19Dec20]"
734775,5fd90a54b8f8002aaabb1b4f,9990010077745197,120771,2021-01-13 03:25:28.820,"{'1607904000': [2, '- -'], '1604620800': [2, '-'], '1593216000': 1, '1589673600': 1, '1588464000': 1, '1609718400': [2, '-']}","{'1609718400': 2, '1607904000': 2, '1604620800': 2}",NaT,NaT,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(38, 21, 38, 21, 38, 29.5, 72.25, 2, 2, 14.75)",1609718400,"[06Nov20, 14Dec20, 04Jan21]"
734776,5fd90a54b8f8002aaabb1fec,9990012171809986,120771,2021-01-13 04:12:31.770,"{'1607904000': [1, '-'], '1607212800': [1, '-'], '1605744000': [1, '-'], '1604275200': [1, '-'], '1602979200': 1, '1609027200': [1, '-'], '1609891200': [1, '-']}","{'1609027200': 1, '1607904000': 1, '1607212800': 1, '1605744000': 1, '1604275200': 1, '1602979200': 1, '1609891200': 1}",NaT,NaT,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(13, 10, 13, 10, 13, 11.5, 2.25, 1, 1, 11.5)",1609891200,"[18Oct20, 02Nov20, 19Nov20, 06Dec20, 14Dec20, 27Dec20, 06Jan21]"
734779,5fd90a54b8f8002aaabb3af8,9990021032212892,120771,2021-01-13 04:01:18.468,"{'1607904000': [1, '-'], '1606348800': [1, '-'], '1604361600': [1, '-'], '1608854400': [1, '-'], '1610064000': [1, '-']}","{'1608854400': 1, '1607904000': 1, '1606348800': 1, '1604361600': 1}",NaT,NaT,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(18, 11, 18, 11, 18, 14.5, 12.25, 1, 1, 14.5)",1608854400,"[03Nov20, 26Nov20, 14Dec20, 25Dec20]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815322,5fdc920bb8f8002aaa243d50,9990030031153247,120771,2021-01-12 13:02:12.907,"{'1608076800': [2, 'T-L20-0628|4400'], '1608854400': [1, '-'], '1609804800': [3, '-']}","{'1608854400': 1, '1608076800': 2, '1609804800': 3}",NaT,NaT,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(9, 11, 11, 9, 9, 10.0, 1.0, 2, 1, 7.75)",1609804800,"[16Dec20, 25Dec20, 05Jan21]"
815612,5fdc94dcb8f8002aaa3dcca9,9990011563134916,120771,2021-01-12 13:11:09.730,"{'1607990400': [1, '-'], '1609459200': [3, '- -'], '1609804800': [1, '-']}","{'1609459200': 3, '1607990400': 1, '1609804800': 1}",NaT,NaT,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(17, 4, 17, 4, 17, 10.5, 42.25, 1, 3, 9.17)",1609804800,"[15Dec20, 01Jan21, 05Jan21]"
815934,5fddb83db8f8002aaa690da0,9990010680227943,120771,2021-01-12 12:46:37.277,"{'1608249600': [1, '-'], '1608854400': [1, '-'], '1609632000': [2, '-']}","{'1609632000': 2, '1608854400': 1, '1608249600': 1}",NaT,NaT,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(7, 9, 9, 7, 7, 8.0, 1.0, 1, 1, 8.0)",1609632000,"[18Dec20, 25Dec20, 03Jan21]"
816342,5fdf1059b8f8002aaa950579,9990012386819389,120771,2021-01-12 12:36:21.130,"{'1608336000': [1, '-'], '1609459200': [1, '-'], '1609891200': [1, '-']}","{'1609459200': 1, '1608336000': 1, '1609891200': 1}",NaT,NaT,SGM EKSPLOR 3+ VANILA BOX 900G,900.0,GR,"(13, 5, 13, 5, 13, 9.0, 16.0, 1, 1, 9

In [21]:
train[(train['mean_qty'] < 7) & (train['variance'] < 70)]

,t_2,t_1,max,min,since,mean,variance,q_2,q_1,mean_qty
11,2,3,3,2,2,2.5,0.25,1,1,2.50
14,5,16,16,5,5,10.5,30.25,1,2,6.50
25,3,3,3,3,3,3.0,0.00,1,1,3.00
39,2,2,2,2,2,2.0,0.00,1,1,2.00
40,3,7,7,3,3,5.0,4.00,5,2,2.05
...,...,...,...,...,...,...,...,...,...,...
9704470,1,6,6,1,1,3.5,6.25,1,1,3.50
9704834,4,1,4,1,4,2.5,2.25,1,1,2.50
9705146,3,1,3,1,3,2.0,1.00,1,1,2.00
9705246,3,3,3,3,3,3.0,0.00,1,1,3.00


In [22]:
# X = train_descp[list(train) + ['UKURAN']].drop(['t_1','q_1'], 1)
# Y = train_descp['t_1']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

# from sklearn.linear_model import LinearRegression
# clf = LinearRegression()
# clf.fit(X_train, y_train)

# train_predictions = clf.predict(X_test)
# from sklearn import metrics
# rmse = np.sqrt( metrics.mean_squared_error( y_test, train_predictions ) )
# print("RMSE: {}".format(rmse))


# test = train_descp[list(train) + ['UKURAN']].drop(['t_5','q_5'], 1)
# test_predictions = clf.predict(test)

# test['next_order'] = list(test_predictions)
# test['next_order'] = test['next_order'].astype(int).abs()

In [23]:
df_plu[df_plu['MPA_DESC'].str.contains('sgm',case=False)]

,MPA_PLU,MPA_DESC,MPA_DESC_SUBCAT
0,120763,SGM EKSPLOR 1+ VANILA BOX 150G,Susu Bubuk
1,120762,SGM EKSPLOR 1+ MADU BOX 150G,Susu Bubuk
7,120773,SGM EKSPLOR 3+ COKLAT BOX 900G,Susu Bubuk
8,120771,SGM EKSPLOR 3+ VANILA BOX 900G,Susu Bubuk
14,210099,SGM EKSPLOR 1+ PLAIN BOX 400G,Susu Bubuk
19,200219,SGM EKSPLOR 5+ MADU BOX 900G,Susu Bubuk
28,120769,SGM EKSPLOR 3+ MADU BOX 400G,Susu Bubuk
32,120764,SGM EKSPLOR 1+ MADU BOX 400G,Susu Bubuk
34,120766,SGM EKSPLOR 1+ MADU BOX 900G,Susu Bubuk
36,210045,SGM EKSPLOR 3+ MADU BOX 150G,Susu Bubuk


In [24]:
train_cluster = pd.concat([train, df_hist['TR_PLU_PCS']], 1)
train_cluster = train_cluster[train_cluster['TR_PLU_PCS'] == 120762].drop(['TR_PLU_PCS','max','min','since','mean'], 1)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=123)
label_p = kmeans.fit_predict(train_cluster)

res_cluster = train_cluster.copy().reset_index(drop=True)
res_cluster['label'] = pd.Series(list(label_p))
dominant = res_cluster['label'].value_counts()[0:1].index[0]

# res_cluster[(res_cluster['label'] == dominant) &\
#             (res_cluster['t_3'] >= 5) & (res_cluster['t_2'] >= 5) & (res_cluster['t_1'] >= 5) &
#             (res_cluster['t_3'] <= 35) & (res_cluster['t_2'] <= 35) & (res_cluster['t_1'] <= 35)].describe()

res_cluster[(res_cluster['label'] == dominant)].describe()

,t_2,t_1,variance,q_2,q_1,mean_qty,label
count,17269.000000,17269.000000,17269.000000,17269.000000,17269.000000,17269.000000,17269.0
mean,21.460826,22.969541,178.940211,1.663906,1.672071,17.999405,0.0
std,20.772682,21.294373,277.228972,1.631633,1.589102,15.125724,0.0
min,1.000000,1.000000,0.000000,1.000000,1.000000,0.090000,0.0
25%,6.000000,6.000000,6.250000,1.000000,1.000000,6.380000,0.0
50%,14.000000,16.000000,49.000000,1.000000,1.000000,13.500000,0.0
75%,31.000000,34.000000,225.000000,2.000000,2.000000,25.670000,0.0
max,116.000000,120.000000,1296.000000,48.000000,34.000000,89.500000,0.0


In [25]:
# train_cluster = pd.concat([train, df_hist['TR_PLU_PCS']], 1)
# train_cluster = train_cluster[train_cluster['TR_PLU_PCS'] == 120762].drop(['TR_PLU_PCS','max','min','since','mean'], 1)

# train_cluster[(train_cluster['t_3'] >= 5) & (train_cluster['t_2'] >= 5) & (train_cluster['t_1'] >= 5) &
#     (train_cluster['t_3'] <= 60) & (train_cluster['t_2'] <= 60) & (train_cluster['t_1'] <= 60)].describe()

In [26]:
# res_cluster[(res_cluster['label'] == 0) & (res_cluster['mean_qty'] >= 7) & (res_cluster['variance'] < 10)]

### Compare time consuming in different netto
> 
* first we user with less variance, we assume user buy based on their consumption and less affected by promotion or random purchase date
* age might be affect consumption, cause more mature, amount of consumption should be decreased by parent

In [27]:
# with open('watchlist_item.txt') as f:
#     watchlist = f.read().splitlines()
#     df_watchlist = pd.DataFrame({'DESCP':watchlist})

In [28]:
train_descp = pd.concat([train,df_hist], 1)[col_feature + ['TR_MEMBERS', 'TR_PLU_PCS', 'DESCP', 'TRF_DATE', 'read_seq', 'UKURAN', 'UOM']]
train_descp = pd.merge(train_descp, df_plu[['MPA_PLU', 'MPA_DESC_SUBCAT']], left_on='TR_PLU_PCS', right_on='MPA_PLU', how='left')
train_descp = train_descp[train_descp['UKURAN'].notnull()]
train_descp['UKURAN'] = train_descp['UKURAN'].astype(int)

In [29]:
# train_descp.groupby('UKURAN').agg({'mean_qty':'mean'})

# train_descp_cond = train_descp[train_descp['variance'] < 100]
# train_descp_cond.groupby('UKURAN').agg({'mean_qty':'mean'})

In [30]:
train_descp['DESCP_TYPE'] = train_descp['DESCP'].str.split(' ').str[0:-1].str.join(' ')

In [31]:
# train_descp_  = train_descp[(train_descp['mean_qty'] >= 6)]

In [32]:
train_descp.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM']).agg({'mean_qty':'median'})

mean_qty
MPA_DESC_SUBCAT        DESCP_TYPE                 UKURAN UOM          
Perawatan Gigi & Mulut PEPSODENT PG ECONOMY       75     GR     29.000
                       PEPSODENT PG ECONOMY JUMBO 190    GR     34.000
                                                  225    GR     25.750
Susu Bubuk             ANLENE ACTIFIT COKELAT BOX 250    GR     25.000
                       ANLENE GOLD COKELAT BOX    250    GR     29.500
                                                  650    GR     33.750
                       ANLENE GOLD ORI BOX        650    GR     33.500
                       ANLENE GOLD+ VANILA BOX    250    GR     29.000
                                                  650    GR     33.000
                       BEBELAC 3 MADU BOX         200    GR     14.170
                                                  400    GR     16.000
                                                  1000   G      19.250
                       BEBELAC 3 MADU CAN         800    GR     17.000
                       BEBELAC 3 VANILA BOX       400    GR     17.000
                                                  1000   G      18.500
                       BEBELAC 3 VANILA CAN       800    GR     17.000
                       BEBELAC 4 MADU BOX         400    GR     17.500
                                                  1000   G      21.000
                       BEBELAC 4 MADU CAN         800    GR     18.500
                       BEBELAC 4 VANILA BOX       400    GR     17.000
                       BEBELAC 4 VANILA CAN       800    GR     18.000
                       BEBELAC GOLD 3 VANILA BOX  700    GR     12.000
                       DANCOW 1+ MADU BOX         200    GR     14.000
                                                  400    GR     18.000
                                                  800    GR     15.000
                       DANCOW 1+ VANILA BOX       400    GR     16.500
                                                  800    GR     15.000
                       DANCOW 3+ COKELAT BOX      400    GR     20.500
                                                  800    GR     18.000
                       DANCOW 3+ MADU BOX         200    GR     15.500
                                                  400    GR     20.000
                                                  800    GR     17.000
                       DANCOW 3+ VANILA BOX       400    GR     20.000
                                                  800    GR     17.250
                       DANCOW 5+ COKELAT BOX      800    GR     21.000
                       DANCOW 5+ MADU BOX         400    GR     21.500
                                                  800    GR     20.000
                       DANCOW FG COKELAT BOX      200    GR     23.000
                                                  400    GR     24.000
                                                  800    GR     22.500
                       DANCOW FG F.CREAM BOX      400    GR     26.500
                                                  800    GR     27.500
                       DANCOW FG INST BOX         200    GR     23.500
                                                  800    GR     24.500
                       FF BABY TAHAP 1 BOX 400G   400    GR     11.000
                       FF BABY TAHAP 1 BOX 800G   800    GR     12.000
                       FF BABY TAHAP 2 BOX 400G   400    GR     11.500
                       FF BABY TAHAP 2 BOX 800G   800    GR     11.500
                       LACTOGROW 3 MADU BOX       180    GR     14.000
                                                  350    GR     17.000
                                                  750    GR     14.250
                       LACTOGROW 3 PLAIN BOX      350    GR     16.625
                                                  750    GR     13.500
                       LACTOGROW 3 VANILA BOX     750    GR     14.250
                       LACTOGROW 4 MADU BOX       750    GR     15.500
               

In [33]:
train_descp_cond = train_descp[(train_descp['variance'] < 400) & (train_descp['mean_qty'] >= 5) & (train_descp['mean_qty'] <= 50)]
train_descp_cond.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM']).agg({'mean_qty':'median'})

mean_qty
MPA_DESC_SUBCAT        DESCP_TYPE                 UKURAN UOM          
Perawatan Gigi & Mulut PEPSODENT PG ECONOMY       75     GR     20.750
                       PEPSODENT PG ECONOMY JUMBO 190    GR     25.000
                                                  225    GR     18.000
Susu Bubuk             ANLENE ACTIFIT COKELAT BOX 250    GR     20.000
                       ANLENE GOLD COKELAT BOX    250    GR     23.000
                                                  650    GR     29.000
                       ANLENE GOLD ORI BOX        650    GR     28.000
                       ANLENE GOLD+ VANILA BOX    250    GR     23.000
                                                  650    GR     27.000
                       BEBELAC 3 MADU BOX         200    GR     13.750
                                                  400    GR     14.000
                                                  1000   G      15.500
                       BEBELAC 3 MADU CAN         800    GR     14.500
                       BEBELAC 3 VANILA BOX       400    GR     14.500
                                                  1000   G      15.500
                       BEBELAC 3 VANILA CAN       800    GR     14.000
                       BEBELAC 4 MADU BOX         400    GR     15.000
                                                  1000   G      17.000
                       BEBELAC 4 MADU CAN         800    GR     16.000
                       BEBELAC 4 VANILA BOX       400    GR     15.000
                       BEBELAC 4 VANILA CAN       800    GR     15.500
                       BEBELAC GOLD 3 VANILA BOX  700    GR     11.750
                       DANCOW 1+ MADU BOX         200    GR     14.000
                                                  400    GR     15.000
                                                  800    GR     13.500
                       DANCOW 1+ VANILA BOX       400    GR     14.500
                                                  800    GR     13.355
                       DANCOW 3+ COKELAT BOX      400    GR     17.000
                                                  800    GR     15.500
                       DANCOW 3+ MADU BOX         200    GR     14.500
                                                  400    GR     16.000
                                                  800    GR     15.000
                       DANCOW 3+ VANILA BOX       400    GR     16.500
                                                  800    GR     15.000
                       DANCOW 5+ COKELAT BOX      800    GR     18.000
                       DANCOW 5+ MADU BOX         400    GR     17.500
                                                  800    GR     17.000
                       DANCOW FG COKELAT BOX      200    GR     17.500
                                                  400    GR     19.000
                                                  800    GR     18.500
                       DANCOW FG F.CREAM BOX      400    GR     21.000
                                                  800    GR     22.000
                       DANCOW FG INST BOX         200    GR     18.000
                                                  800    GR     20.500
                       FF BABY TAHAP 1 BOX 400G   400    GR     11.500
                       FF BABY TAHAP 1 BOX 800G   800    GR     11.500
                       FF BABY TAHAP 2 BOX 400G   400    GR     12.000
                       FF BABY TAHAP 2 BOX 800G   800    GR     11.000
                       LACTOGROW 3 MADU BOX       180    GR     14.500
                                                  350    GR     14.500
                                                  750    GR     12.500
                       LACTOGROW 3 PLAIN BOX      350    GR     14.500
                                                  750    GR     12.500
                       LACTOGROW 3 VANILA BOX     750    GR     12.600
                       LACTOGROW 4 MADU BOX       750    GR     14.000
               

In [34]:
from ipywidgets import interact, interact_manual
@interact_manual
def show_articles_more_than(nama = sorted(list(train_descp['DESCP_TYPE'].unique())), 
                            ukuran = sorted(list(train_descp['UKURAN'].unique())),
                            lower=(2, 60, 2),
                            upper=(2, 60, 2)):
    show_data = train_descp.loc[(train_descp['DESCP_TYPE'] == nama) & (train_descp['UKURAN'] == ukuran)\
                                & (train_descp['mean_qty'] >= lower) & (train_descp['mean_qty'] <= upper)]\
                .drop(['since','mean','TRF_DATE','MPA_PLU','MPA_DESC_SUBCAT','DESCP_TYPE'],1)
    print(show_data.shape)
    sample_n = 10
    if show_data.shape[0] < sample_n:
        sample_n = show_data.shape[0]
    return show_data.sample(sample_n)

interactive(children=(Dropdown(description='nama', options=('ANLENE ACTIFIT COKELAT BOX', 'ANLENE GOLD COKELAT…

In [41]:
bins = [0, 7, 14, 21, 28, 35, 42]
labels = ['0-7','7-14','14-21','21-28','28-35','35-42']
train_descp['mean_qty_binned'] = pd.cut(train_descp['mean_qty'], bins=bins, labels=labels, include_lowest=True)

In [64]:
train_descp_select1 = train_descp[train_descp['DESCP'].str.startswith('SGM EKSPLOR 1+ VANILA')]
train_descp_select1.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM','mean_qty_binned']).agg({'mean_qty_binned':'size'})

mean_qty_binned
MPA_DESC_SUBCAT DESCP_TYPE                UKURAN UOM mean_qty_binned                 
Susu Bubuk      SGM EKSPLOR 1+ VANILA BOX 150    GR  0-7                         2573
                                                     7-14                        2337
                                                     14-21                       1707
                                                     21-28                       1200
                                                     28-35                        932
                                                     35-42                        646
                                          400    GR  0-7                         2424
                                                     7-14                        2803
                                                     14-21                       2140
                                                     21-28                       1490
                                                     28-35                       1065
                                                     35-42                        806
                                          600    GR  0-7                         1889
                                                     7-14                        2800
                                                     14-21                       2245
                                                     21-28                       1752
                                                     28-35                       1307
                                                     35-42                        954
                                          900    GR  0-7                         5190
                                                     7-14                        8844
                                                     14-21                       6247
                                                     21-28                       4316
                                                     28-35                       3057
                                                     35-42                       2110

In [67]:
train_descp_select1 = train_descp[train_descp['DESCP'].str.startswith('SGM EKSPLOR 1+ MADU')]
train_descp_select1.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM','mean_qty_binned']).agg({'mean_qty_binned':'size'})

mean_qty_binned
MPA_DESC_SUBCAT DESCP_TYPE              UKURAN UOM mean_qty_binned                 
Susu Bubuk      SGM EKSPLOR 1+ MADU BOX 150    GR  0-7                         4949
                                                   7-14                        4033
                                                   14-21                       2861
                                                   21-28                       1987
                                                   28-35                       1483
                                                   35-42                       1077
                                        400    GR  0-7                         4258
                                                   7-14                        4903
                                                   14-21                       3444
                                                   21-28                       2478
                                                   28-35                       1816
                                                   35-42                       1394
                                        600    GR  0-7                         3946
                                                   7-14                        5839
                                                   14-21                       4582
                                                   21-28                       3307
                                                   28-35                       2650
                                                   35-42                       1884
                                        900    GR  0-7                         8711
                                                   7-14                       13925
                                                   14-21                       9418
                                                   21-28                       6057
                                                   28-35                       4449
                                                   35-42                       3045

In [35]:
all_item = pd.merge(df_plu, df_nas, left_on='MPA_PLU', right_on='ITEM_N')[['MPA_PLU', 'MPA_DESC', 'UKURAN']]
all_item = all_item.sort_values('MPA_DESC')

data_filter = []
for idx, row in all_item.iterrows():
    
    show_data = train_descp.loc[(train_descp['TR_PLU_PCS'] == row['MPA_PLU']) & (train_descp['UKURAN'] == row['UKURAN'])]

    

    Q1 = round(show_data['mean_qty'].quantile(0.25),2)
    Q2 = round(show_data['mean_qty'].quantile(0.5),2)
    Q3 = round(show_data['mean_qty'].quantile(0.75),2)
    IQR= round(Q3 - Q1, 2)
    upper_out = Q3 + 1.5 * IQR
    lower_out = Q1 - 1.5 * IQR
    MAX = show_data['mean_qty'].max()
    MIN = show_data['mean_qty'].min()
    
    # calculate summary statistics
    data_mean, data_std = show_data['mean_qty'].mean(), show_data['mean_qty'].std()
    # identify outliers
    cut_off = data_std * 1
    lower, upper = data_mean - cut_off, data_mean + cut_off
#     lower = 7
#     upper = 30



    show_data_ = show_data[(show_data['mean_qty'] > lower) & (show_data['mean_qty'] < upper)]
    data_filter.append(show_data_)
    median = round(show_data_['mean_qty'].median(),2)
    
#     print("Q1:{} Q2:{} median:{} Q3:{} IQR:{} MAX:{} MIN:{}".format(Q1,Q2,median,Q3,IQR,MAX,MIN))
#     print("lower {} upper{}".format(lower, upper))
#     print()
    
show_data_filter = pd.concat(data_filter)

In [36]:
show_data_filter.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM']).agg({'mean_qty':'median'})

mean_qty
MPA_DESC_SUBCAT        DESCP_TYPE                 UKURAN UOM          
Perawatan Gigi & Mulut PEPSODENT PG ECONOMY       75     GR     29.000
                       PEPSODENT PG ECONOMY JUMBO 190    GR     34.000
                                                  225    GR     26.500
Susu Bubuk             ANLENE ACTIFIT COKELAT BOX 250    GR     24.500
                       ANLENE GOLD COKELAT BOX    250    GR     29.000
                                                  650    GR     33.500
                       ANLENE GOLD ORI BOX        650    GR     33.000
                       ANLENE GOLD+ VANILA BOX    250    GR     28.500
                                                  650    GR     32.500
                       BEBELAC 3 MADU BOX         200    GR     12.830
                                                  400    GR     14.500
                                                  1000   G      18.000
                       BEBELAC 3 MADU CAN         800    GR     15.500
                       BEBELAC 3 VANILA BOX       400    GR     16.000
                                                  1000   G      17.500
                       BEBELAC 3 VANILA CAN       800    GR     15.290
                       BEBELAC 4 MADU BOX         400    GR     16.355
                                                  1000   G      20.500
                       BEBELAC 4 MADU CAN         800    GR     17.500
                       BEBELAC 4 VANILA BOX       400    GR     16.000
                       BEBELAC 4 VANILA CAN       800    GR     16.500
                       BEBELAC GOLD 3 VANILA BOX  700    GR     10.500
                       DANCOW 1+ MADU BOX         200    GR     12.000
                                                  400    GR     17.000
                                                  800    GR     13.500
                       DANCOW 1+ VANILA BOX       400    GR     15.000
                                                  800    GR     13.500
                       DANCOW 3+ COKELAT BOX      400    GR     19.500
                                                  800    GR     16.750
                       DANCOW 3+ MADU BOX         200    GR     14.000
                                                  400    GR     18.500
                                                  800    GR     15.500
                       DANCOW 3+ VANILA BOX       400    GR     19.500
                                                  800    GR     16.000
                       DANCOW 5+ COKELAT BOX      800    GR     20.500
                       DANCOW 5+ MADU BOX         400    GR     21.000
                                                  800    GR     18.500
                       DANCOW FG COKELAT BOX      200    GR     22.000
                                                  400    GR     23.500
                                                  800    GR     21.750
                       DANCOW FG F.CREAM BOX      400    GR     26.125
                                                  800    GR     27.000
                       DANCOW FG INST BOX         200    GR     22.500
                                                  800    GR     24.000
                       FF BABY TAHAP 1 BOX 400G   400    GR      9.500
                       FF BABY TAHAP 1 BOX 800G   800    GR     10.750
                       FF BABY TAHAP 2 BOX 400G   400    GR      9.500
                       FF BABY TAHAP 2 BOX 800G   800    GR     10.500
                       LACTOGROW 3 MADU BOX       180    GR     12.250
                                                  350    GR     15.500
                                                  750    GR     12.500
                       LACTOGROW 3 PLAIN BOX      350    GR     15.500
                                                  750    GR     11.750
                       LACTOGROW 3 VANILA BOX     750    GR     12.500
                       LACTOGROW 4 MADU BOX       750    GR     14.000
               